# Regresión Lineal Múltiple — Dataset FuelConsumptionCO2

Esta notebook explica, paso a paso y en lenguaje claro, cómo construir un modelo de **regresión lineal múltiple** para predecir **emisiones de CO2** usando variables de consumo y motor de un conjunto de datos real del Gobierno de Canadá.

Objetivos:
- Entender qué es la regresión lineal múltiple y para qué sirve.
- Cargar un dataset en línea y seleccionar variables útiles.
- Dividir datos en entrenamiento y prueba con `train_test_split` (scikit-learn).
- Entrenar un modelo con `LinearRegression` y entender sus coeficientes.
- Evaluar el modelo con métricas (R2, MAE, RMSE) y explicar su significado.
- Visualizar el **plano de regresión** en 3D y entender visualmente el ajuste.


## 1. ¿Qué es la regresión lineal múltiple?

Es un modelo que relaciona una variable objetivo (en este caso, **CO2EMISSIONS**) con **dos o más** variables explicativas (por ejemplo, **ENGINESIZE** y **FUELCONSUMPTION_COMB**). La idea es encontrar un **plano** (en 3D) o **hiperplano** (si hay más de dos predictores) que se ajuste a los datos de tal forma que las predicciones estén lo más cerca posible de los valores reales.

Cómo se entrena: se eligen los parámetros (intercepto y coeficientes) que **minimizan el error cuadrático medio** entre lo observado y lo predicho. Ese procedimiento se conoce como **Mínimos Cuadrados Ordinarios (OLS)**. La cercanía entre el plano y cada punto se mide con el **residuo** (observado menos predicho).

## 2. Cargar el dataset en línea y explorar

Usaremos el archivo `FuelConsumptionCo2.csv`. Este dataset contiene, entre otras, las variables:
- ENGINESIZE: tamaño del motor (litros)
- FUELCONSUMPTION_COMB: consumo de combustible combinado (L/100km)
- CO2EMISSIONS: emisiones de CO2 (g/km)

Comenzaremos cargando el archivo y mirando sus primeras filas.

In [ ]:
import pandas as pd
import numpy as np

url = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/ML0101ENv3/labs/FuelConsumptionCo2.csv'
df = pd.read_csv(url)
df.head()

Seleccionamos columnas relevantes para el ejemplo. Puedes añadir más predictores (por ejemplo, `CYLINDERS`) y comparar resultados.

In [ ]:
cdf = df[['ENGINESIZE', 'FUELCONSUMPTION_COMB', 'CO2EMISSIONS']].dropna()
cdf.describe()

## 3. Visualización simple

Antes de modelar, siempre es buena idea observar las relaciones básicas. Aquí vamos a graficar **ENGINESIZE** vs **CO2EMISSIONS** y **FUELCONSUMPTION_COMB** vs **CO2EMISSIONS** para tener una idea de su relación.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.scatter(cdf['ENGINESIZE'], cdf['CO2EMISSIONS'])
plt.xlabel('ENGINESIZE (L)')
plt.ylabel('CO2EMISSIONS (g/km)')
plt.title('Motor vs CO2')

plt.subplot(1,2,2)
plt.scatter(cdf['FUELCONSUMPTION_COMB'], cdf['CO2EMISSIONS'])
plt.xlabel('FUELCONSUMPTION_COMB (L/100km)')
plt.ylabel('CO2EMISSIONS (g/km)')
plt.title('Consumo vs CO2')
plt.tight_layout()
plt.show()

## 4. División en entrenamiento y prueba con `train_test_split`

Para evaluar de forma honesta un modelo, separamos los datos en dos partes:
- **Entrenamiento**: el modelo aprende los coeficientes.
- **Prueba**: el modelo se evalúa en datos que no vio durante el entrenamiento.

Usaremos `train_test_split` de scikit-learn, que es la forma recomendada: permite controlar el tamaño del conjunto de prueba, la aleatoriedad y la reproducibilidad con `random_state`.


In [ ]:
from sklearn.model_selection import train_test_split

X = cdf[['ENGINESIZE', 'FUELCONSUMPTION_COMB']]
y = cdf['CO2EMISSIONS']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 20% para prueba
    random_state=42,    # semilla para reproducibilidad
    shuffle=True        # mezclar aleatoriamente antes de dividir
)

len(X_train), len(X_test)

## 5. Entrenamiento del modelo con `LinearRegression`

Entrenaremos un modelo de regresión lineal múltiple utilizando `LinearRegression`. Este algoritmo calcula el intercepto y los coeficientes que minimizan el error cuadrático medio en el conjunto de entrenamiento.

In [ ]:
from sklearn.linear_model import LinearRegression

modelo = LinearRegression(fit_intercept=True)
modelo.fit(X_train, y_train)

intercepto = modelo.intercept_
coeficientes = modelo.coef_
nombres = X_train.columns

print('Intercepto (a):', round(intercepto, 3))
print('Coeficientes (b):')
for nombre, b in zip(nombres, coeficientes):
    print(f'  {nombre}: {round(b, 4)}')

Cómo interpretar estos números:
- El **intercepto** es el valor predicho de CO2EMISSIONS cuando todas las variables son cero (no siempre tiene sentido físico, pero es parte del modelo).
- Cada **coeficiente** indica cuánto se espera que cambie CO2EMISSIONS cuando esa variable aumenta una unidad **manteniendo las otras constantes**. Por ejemplo, si el coeficiente de ENGINESIZE es 40, entonces al aumentar 1 litro en el tamaño del motor, se esperarían +40 g/km en emisiones, manteniendo el consumo constante.

## 6. Predicción y métricas de evaluación

Ahora predecimos en el conjunto de prueba y calculamos métricas:
- **R2**: proporción de la variación en el objetivo explicada por el modelo (0 a 1; más alto es mejor).
- **MAE**: error absoluto medio (promedio de |observado - predicho|) en unidades de la variable objetivo.
- **RMSE**: raíz del error cuadrático medio; es como un "error típico" también en unidades del objetivo.

In [ ]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np

y_pred = modelo.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print(f'R2:  {r2:.3f}')
print(f'MAE: {mae:.3f} g/km')
print(f'RMSE:{rmse:.3f} g/km')

Cómo leer estas métricas:
- **R2** cercano a 1 indica que el modelo captura bien la variación del objetivo; cerca de 0 indica que explica poco.
- **MAE** y **RMSE** más pequeños indican predicciones más cercanas a los valores reales. RMSE penaliza más los errores grandes que el MAE.

Tip: si añades más variables, compara las métricas para ver si el modelo mejora de forma consistente (mejor R2 y menores MAE/RMSE).

## 7. Visualización 3D del plano de regresión

Un plano de regresión en 3D ayuda a entender visualmente cómo el modelo combina **ENGINESIZE** y **FUELCONSUMPTION_COMB** para predecir **CO2EMISSIONS**. Los puntos son los datos reales; el plano es lo que predice el modelo. La distancia vertical de cada punto al plano es el **residuo** (observado - predicho). Cuanto más pequeños son esos residuos en promedio, mejor el ajuste.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 (necesario para proyección 3D)
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(9,7))
ax = fig.add_subplot(111, projection='3d')

ax.scatter(cdf['ENGINESIZE'], cdf['FUELCONSUMPTION_COMB'], cdf['CO2EMISSIONS'])

eng_grid, fuel_grid = np.meshgrid(
    np.linspace(cdf['ENGINESIZE'].min(), cdf['ENGINESIZE'].max(), 30),
    np.linspace(cdf['FUELCONSUMPTION_COMB'].min(), cdf['FUELCONSUMPTION_COMB'].max(), 30)
)
co2_grid = modelo.intercept_ + modelo.coef_[0]*eng_grid + modelo.coef_[1]*fuel_grid

ax.plot_surface(eng_grid, fuel_grid, co2_grid, alpha=0.5)
ax.set_xlabel('ENGINESIZE (L)')
ax.set_ylabel('FUELCONSUMPTION_COMB (L/100km)')
ax.set_zlabel('CO2EMISSIONS (g/km)')
ax.set_title('Plano de regresión lineal múltiple')
plt.show()

## 8. Conclusiones didácticas

- La regresión lineal múltiple encuentra un plano que minimiza la suma de los cuadrados de los residuos.
- Los **coeficientes** indican el efecto promedio de cada predictor manteniendo los demás constantes.
- Las **métricas** (R2, MAE, RMSE) sirven para evaluar qué tan bien el modelo generaliza a datos nuevos.
- La visualización 3D ayuda a comunicar la idea de un plano de ajuste y a identificar posibles patrones no lineales o dispersiones heterogéneas.
